In [1]:
from training.perceiver import*
from training.utils import*
from training.losses import*
from training.VIT import*
from training.ResNet import*
from collections import defaultdict
from training import*

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

import torch
import numpy as np
from torch import nn, einsum
import torch.nn.functional as F
import einops as einops
from einops import rearrange, repeat
from einops.layers.torch import Reduce

#import matplotlib.pyplot as plt

import cProfile, pstats, io
from pstats import SortKey

torch.manual_seed(0)

In [2]:
import numpy as np
ids=get_ids_in_folder(base_path="./data/public/1.0.1/test/")

idx=10
data=load_npz_by_id(idx,base_path="./data/public/1.0.1/test/")



#data=np.load("path")
tmp_name=data["alos"][idx] #shape : 5000;4;4;4;3 (batch / time / h / w / c)
print(tmp_name.shape)

#3456
#11520
#1920
#420
#192



(4, 4, 4, 3)


In [3]:
#compute_channel_mean_std(mode="train")

In [4]:
#create_dataset(name="tiny", mode="train",max_imgs=-1)
#create_dataset(name="tiny", mode="test",max_imgs=-1)
#create_dataset(name="tiny", mode="validation",max_imgs=-1)

In [5]:
config=read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
transconfig=transformations_config_flair("./data/bands_info/bands.yaml",config)

dataset=CustomPlanted("./data/custom_planted/tiny_train.h5", config=config, trans_config=transconfig)

In [6]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import torch
import os

seed_everything(42, workers=True)

torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')

config_model = read_yaml("./training/configs/config_test-Atomiser_Atos.yaml")
labels = load_json_to_dict("./data/labels.json")
bands_yaml = "./data/bands_info/bands.yaml"

trans_config = transformations_config_flair(bands_yaml, config_model)
xp_name = "test_Atos"

wand = False
wandb_logger = None
if wand:
    if os.environ.get("LOCAL_RANK", "0") == "0":
        import wandb
        wandb.init(
            name=get_xp_name(config_model['encoder']) + " modalities",
            project="Atos_modalities",
            config=config_model
        )
        wandb_logger = WandbLogger(project="tiny_modalities")

model = Model(config_model, wand=wand, name=xp_name, labels=labels,transform=trans_config)

data_module = CustomPlantedDataModule(
    "./data/custom_planted/tiny",
    config=config_model,
    trans_config=trans_config,
    batch_size=config_model['dataset']['batchsize'],
)

data_module.setup()


# Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints/",
    filename="best_model-{epoch:02d}-{val_F1:.4f}",
    monitor="val_F1",
    mode="max",
    save_top_k=1,
    verbose=True,
)

early_stop_callback = EarlyStopping(
    monitor="val_F1",
    min_delta=0.00,
    patience=15,
    verbose=False,
    mode="max"
)



# Trainer
trainer = Trainer(
    use_distributed_sampler=False,
    #strategy="ddp",
    #devices=-1,
    max_epochs=config_model["trainer"]["epochs"],
    logger=wandb_logger,
    log_every_n_steps=1,
    accelerator="gpu",
    callbacks=[early_stop_callback, checkpoint_callback],
    default_root_dir="./checkpoints/",
    limit_train_batches=0.001,
    limit_val_batches=0.001
)

pr = cProfile.Profile()
pr.enable()
# Fit the model
trainer.fit(model, datamodule=data_module)

pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())


Seed set to 42


NameError: name 'get_input_attn' is not defined